# 08 pandas 数据分组

分组统计 - groupby功能

* 根据某些条件将数据拆分成组
* 对每个组独立应用函数
* 将结果合并到一个数据结构中

Dataframe在行（axis=0）或列（axis=1）上进行分组，将一个函数应用到各个分组并产生一个新值，然后函数执行结果被合并到最终的结果对象中。

df.groupby(by=None, axis=0, level=None, as_index=True, sort=True, group_keys=True, squeeze=False, **kwargs)

## 基本内容

In [1]:
# 导入numpy、pandas模块
import numpy as np
import pandas as pd  

In [2]:
# 分组

df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar','foo', 'bar', 'foo', 'foo'],
                   'B' : ['one', 'one', 'two', 'three', 'two', 'two', 'one', 'three'],
                   'C' : np.random.randn(8),
                   'D' : np.random.randn(8)})
print(df)
print('------')

print(df.groupby('A'), type(df.groupby('A')))
print('------')
# 直接分组得到一个groupby对象，是一个中间数据，没有进行计算

a = df.groupby('A').mean()
b = df.groupby(['A','B']).mean()
c = df.groupby(['A'])['D'].mean()  # 以A分组，算D的平均值
print(a,type(a),'\n',a.columns)
print(b,type(b),'\n',b.columns)
print(c,type(c))
# 通过分组后的计算，得到一个新的dataframe
# 默认axis = 0，以行来分组
# 可单个或多个（[]）列分组

     A      B         C         D
0  foo    one  0.257411  0.759321
1  bar    one -1.562436 -1.082005
2  foo    two -1.069473 -0.250684
3  bar  three -1.511564 -1.058422
4  foo    two  0.084918  0.604193
5  bar    two  1.554611 -0.326090
6  foo    one -0.215614  0.188660
7  foo  three -0.309367 -0.729371
------
<pandas.core.groupby.generic.DataFrameGroupBy object at 0x000002A259F1AAC8> <class 'pandas.core.groupby.generic.DataFrameGroupBy'>
------
            C         D
A                      
bar -0.506463 -0.822172
foo -0.250425  0.114424 <class 'pandas.core.frame.DataFrame'> 
 Index(['C', 'D'], dtype='object')
                  C         D
A   B                        
bar one   -1.562436 -1.082005
    three -1.511564 -1.058422
    two    1.554611 -0.326090
foo one    0.020899  0.473990
    three -0.309367 -0.729371
    two   -0.492277  0.176755 <class 'pandas.core.frame.DataFrame'> 
 Index(['C', 'D'], dtype='object')
A
bar   -0.822172
foo    0.114424
Name: D, dtype: float64 <class 

In [3]:
# 分组 - 可迭代对象

df = pd.DataFrame({'X' : ['A', 'B', 'A', 'B'], 'Y' : [1, 4, 3, 2]})
print(df)
print(df.groupby('X'), type(df.groupby('X')))
print('-----')

print(list(df.groupby('X')), '→ 可迭代对象，直接生成list\n')
print(list(df.groupby('X'))[0], '→ 以元祖形式显示\n')
for n,g in df.groupby('X'):
    print(n)
    print(g)
    print('###')
print('-----')
# n是组名，g是分组后的Dataframe

print(df.groupby(['X']).get_group('A'),'\n')
print(df.groupby(['X']).get_group('B'),'\n')
print('-----')
# .get_group()提取分组后的组

grouped = df.groupby(['X'])
print(grouped.groups)
print(grouped.groups['A'])  # 也可写：df.groupby('X').groups['A']
print('-----')
# .groups：将分组后的groups转为dict
# 可以字典索引方法来查看groups里的元素

sz = grouped.size()
print(sz,type(sz))
print('-----')
# .size()：查看分组后的长度

df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar','foo', 'bar', 'foo', 'foo'],
                   'B' : ['one', 'one', 'two', 'three', 'two', 'two', 'one', 'three'],
                   'C' : np.random.randn(8),
                   'D' : np.random.randn(8)})
grouped = df.groupby(['A','B']).groups
print(df)
print(grouped)
print(grouped[('foo', 'three')])
# 按照两个列进行分组

   X  Y
0  A  1
1  B  4
2  A  3
3  B  2
<pandas.core.groupby.generic.DataFrameGroupBy object at 0x000002A259D04978> <class 'pandas.core.groupby.generic.DataFrameGroupBy'>
-----
[('A',    X  Y
0  A  1
2  A  3), ('B',    X  Y
1  B  4
3  B  2)] → 可迭代对象，直接生成list

('A',    X  Y
0  A  1
2  A  3) → 以元祖形式显示

A
   X  Y
0  A  1
2  A  3
###
B
   X  Y
1  B  4
3  B  2
###
-----
   X  Y
0  A  1
2  A  3 

   X  Y
1  B  4
3  B  2 

-----
{'A': Int64Index([0, 2], dtype='int64'), 'B': Int64Index([1, 3], dtype='int64')}
Int64Index([0, 2], dtype='int64')
-----
X
A    2
B    2
dtype: int64 <class 'pandas.core.series.Series'>
-----
     A      B         C         D
0  foo    one  2.839053  0.344213
1  bar    one -0.020621 -0.389806
2  foo    two  0.802460 -2.060197
3  bar  three  0.371008 -0.838587
4  foo    two  0.853277  0.560154
5  bar    two -0.368770  0.294617
6  foo    one  0.363951  0.161507
7  foo  three -0.028561 -0.533982
{('bar', 'one'): Int64Index([1], dtype='int64'), ('bar', 'three'): Int64Inde

In [4]:
# 分组计算函数方法

s = pd.Series([1, 2, 3, 10, 20, 30], index = [1, 2, 3, 1, 2, 3])
grouped = s.groupby(level=0)  # 唯一索引用.groupby(level=0)，将同一个index的分为一组
print(grouped)
print(grouped.first(),'→ first：非NaN的第一个值\n')
print(grouped.last(),'→ last：非NaN的最后一个值\n')
print(grouped.sum(),'→ sum：非NaN的和\n')
print(grouped.mean(),'→ mean：非NaN的平均值\n')
print(grouped.median(),'→ median：非NaN的算术中位数\n')
print(grouped.count(),'→ count：非NaN的值\n')
print(grouped.min(),'→ min、max：非NaN的最小值、最大值\n')
print(grouped.std(),'→ std，var：非NaN的标准差和方差\n')
print(grouped.prod(),'→ prod：非NaN的积\n')

1    1
2    2
3    3
dtype: int64 → first：非NaN的第一个值

1    10
2    20
3    30
dtype: int64 → last：非NaN的最后一个值

1    11
2    22
3    33
dtype: int64 → sum：非NaN的和

1     5.5
2    11.0
3    16.5
dtype: float64 → mean：非NaN的平均值

1     5.5
2    11.0
3    16.5
dtype: float64 → median：非NaN的算术中位数

1    2
2    2
3    2
dtype: int64 → count：非NaN的值

1    1
2    2
3    3
dtype: int64 → min、max：非NaN的最小值、最大值

1     6.363961
2    12.727922
3    19.091883
dtype: float64 → std，var：非NaN的标准差和方差

1    10
2    40
3    90
dtype: int64 → prod：非NaN的积



In [5]:
# 多函数计算：agg()

import warnings                          # 清除警告
warnings.filterwarnings('ignore')


df = pd.DataFrame({'a':[1,1,2,2],
                  'b':np.random.rand(4),
                  'c':np.random.rand(4),
                  'd':np.random.rand(4),})
print(df)
print(df.groupby('a').agg(['mean',np.sum]))
print(df.groupby('a')['b'].agg({'result1':np.mean,
                               'result2':np.sum}))
# 函数写法可以用str，或者np.方法
# 可以通过list，dict传入，当用dict时，key名为columns → 更新pandas后会出现警告
# 尽量用list传入

   a         b         c         d
0  1  0.410724  0.699705  0.708194
1  1  0.313873  0.562457  0.163839
2  2  0.027440  0.190490  0.038462
3  2  0.975547  0.006164  0.696456
          b                   c                   d          
       mean       sum      mean       sum      mean       sum
a                                                            
1  0.362298  0.724597  0.631081  1.262162  0.436016  0.872032
2  0.501493  1.002987  0.098327  0.196654  0.367459  0.734918
    result1   result2
a                    
1  0.362298  0.724597
2  0.501493  1.002987


## 作业

### 作业1：按要求创建Dataframe df，并通过分组得到以下结果
* 以A分组，求出C,D的分组平均值
* 以A,B分组，求出D,E的分组求和
* 以A分组，得到所有分组，以字典显示
* 按照数值类型分组，求和
* 将C,D作为一组分出来，并计算求和
* 以B分组，求出每组的均值，求和，最大值，最小值

In [6]:
df = pd.DataFrame({'A':['one','two','three','one','two','three','one','two'],
                  'B':list('hhhhffff'),
                  'C':list(range(10,25,2)),
                  'D':np.random.randn(8),
                  'E':np.random.randn(8)})
df

,A,B,C,D,E
0,one,h,10,1.215423,-0.052336
1,two,h,12,1.944799,-1.281985
2,three,h,14,0.184295,-0.426786
3,one,h,16,-1.788256,0.219310
4,two,f,18,0.815966,-0.195950
5,three,f,20,-2.465773,1.022250
6,one,f,22,0.322966,1.636517
7,two,f,24,-0.339621,-0.892763


In [7]:
df.groupby(by = 'A')[['D','E']].mean()

,D,E
A,,
one,-0.083289,0.601164
three,-1.140739,0.297732
two,0.807048,-0.790233


In [8]:
df.groupby(['A','B'])[['D','E']].sum()

D         E
A     B                    
one   f  0.322966  1.636517
      h -0.572833  0.166974
three f -2.465773  1.022250
      h  0.184295 -0.426786
two   f  0.476345 -1.088713
      h  1.944799 -1.281985

In [9]:
df.groupby('A').groups

{'one': Int64Index([0, 3, 6], dtype='int64'),
 'three': Int64Index([2, 5], dtype='int64'),
 'two': Int64Index([1, 4, 7], dtype='int64')}

In [10]:
g = df.groupby('A')
for n, dfi in g:
    print(n)
    print(dfi)

one
     A  B   C         D         E
0  one  h  10  1.215423 -0.052336
3  one  h  16 -1.788256  0.219310
6  one  f  22  0.322966  1.636517
three
       A  B   C         D         E
2  three  h  14  0.184295 -0.426786
5  three  f  20 -2.465773  1.022250
two
     A  B   C         D         E
1  two  h  12  1.944799 -1.281985
4  two  f  18  0.815966 -0.195950
7  two  f  24 -0.339621 -0.892763


In [11]:
df.groupby(df.dtypes, axis=1).sum()

,int64,float64,object
0,10,1.163087,oneh
1,12,0.662814,twoh
2,14,-0.242491,threeh
3,16,-1.568946,oneh
4,18,0.620015,twof
5,20,-1.443523,threef
6,22,1.959483,onef
7,24,-1.232383,twof


In [12]:
df2 = df[['C','D']]
df2['sum']=df2.sum(axis=1)
df2

,C,D,sum
0,10,1.215423,11.215423
1,12,1.944799,13.944799
2,14,0.184295,14.184295
3,16,-1.788256,14.211744
4,18,0.815966,18.815966
5,20,-2.465773,17.534227
6,22,0.322966,22.322966
7,24,-0.339621,23.660379


In [13]:
df.groupby('B').agg(['mean','sum','max','min'])

C                     D                                       E  \
  mean sum max min      mean       sum       max       min      mean   
B                                                                      
f   21  84  24  18 -0.416616 -1.666462  0.815966 -2.465773  0.392514   
h   13  52  16  10  0.389065  1.556260  1.944799 -1.788256 -0.385449   

                                 
        sum       max       min  
B                                
f  1.570054  1.636517 -0.892763  
h -1.541796  0.219310 -1.281985